# LangChain demo

## Links
1. [samwit - colab - YT LangChain - Agents.ipynb, React & Serpai ](https://colab.research.google.com/drive/1QpvUHQzpHPvlMgBElwd5NJQ6qpryVeWE?usp=sharing)
1. [samwit - github youtube samples](https://github.com/samwit/langchain-tutorials/tree/main)
1. [Building Custom Tools and Agents with LangChain (gpt-3.5-turbo)](https://www.youtube.com/watch?v=biS8G8x8DdA): [samwit - colab - YT LangChain Custom Tools & Agents.ipynb](https://colab.research.google.com/drive/1FYsa3x3PzziL57EHEIuIqa5rkCAxCbin?usp=sharing) uses chat-conversational-react-description

## Demo use case

### Tools
1. IncidentTool. Simple text
1. KnowledgeBaseTool: Vectorstore 
1. NewRelicTool: PandasAI
1. OrgSearchTool: Team, people, rules of engagement

### Flow
1. Get inc
1. Describe steps for user to create/trigger
    1. Narrative
    1. Bullet point flow
1. Describe what happened
    1. Narrative
    1. Bullet point flow
1. Team to direct inc to
    1. Team/person
    1. How to engage (i.e., email, Absa Snow with configs)
1. Extraction

In [29]:
import os
import json

# Load setting from Json outside of project.
f = open('../../env/ai.json')
settingsJson = json.load(f)
del f

for key in settingsJson:
    os.environ[key] = settingsJson[key]
    
del settingsJson
    
# # OR manually set them
# os.environ['REQUESTS_CA_BUNDLE'] = '../../env/ZCert.pem'
# os.environ['HUGGING_FACE_API_KEY'] = 'Get here: https://huggingface.co/settings/tokens'
# os.environ['OPENAI_API_KEY'] = 'Get here: https://platform.openai.com/account/api-keys'
# os.environ["SERPAPI_API_KEY"] = 'serpapi KEY, Get here: https://serpapi.com/manage-api-key'


In [30]:
#!pip install langchain unstructured openai chromadb Cython tiktoken pypdf


# Old code
        #!pip install unstructured
        # I had issues here so ran the following to make PC trust PIP
        # pip install --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host files.pythonhosted.org pip setuptools
        # pip config set global.trusted-host "pypi.org files.pythonhosted.org pypi.python.org"

        #!pip install markdown
        #!pip install urllib3==1.25.11

        #!pip install pypdf
        
        #!pip install chromadb
        # For chromadb you have to have C++ builders intalled see link below: "I have faced similar issue on Windows OS, while doing..."
        #   https://stackoverflow.com/questions/73969269/error-could-not-build-wheels-for-hnswlib-which-is-required-to-install-pyprojec

        #!pip install pydantic-settings

In [31]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# Tools 
from langchain.agents import Tool
from langchain.tools import BaseTool

In [32]:
llm = OpenAI(temperature=0)

## Create custom tool

# Simple Text Tool
Returns simple text

In [33]:
# Simple Text Tool

def incident_search(incident_number: str):    
    with open('data/incident-store.txt') as f:
        lines = f.readlines()
    
    return  lines

incident_search_tool = Tool(
    name='Incident tool', 
    func= incident_search,    
    description="""
        Useful for queries about incidents.         
        Use the date to order incidents.
        The 'State' of an incident is 'Open' when the State field is not 'Resolved', 'Closed', 'Cancelled'        
        """
)

# Vectorstore 
Vectorstore
https://python.langchain.com/docs/integrations/toolkits/vectorstore \
\
Markdown \
Try this later on \
https://python.langchain.com/docs/modules/data_connection/document_loaders/markdown \
https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/markdown_header_metadata 



In [34]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/knowledge-base.pdf")
internalKbPages = loader.load_and_split()

# for i in pages:
#     print(i)
#     print('**************************')

# TODO: add a publicKnowledgePages collection
# loader = PyPDFLoader("data/south-africa-sarb-currency-and-exchanges-guidelines-for-business-entities.pdf")
# publicKnowledgePages = loader.load_and_split()

# loader = PyPDFLoader("data/south-africa-sarb-currency-and-exchanges-guidelines-for-business-entities.pdf")
# publicKnowledgePages.append(loader.load_and_split())

# loader = PyPDFLoader("data/south-africa-currency-and-exchanges-manual-for-authorised-dealers.pdf")
# publicKnowledgePages.append(loader.load_and_split())

# loader = PyPDFLoader("data/fx-global-code.pdf")
# publicKnowledgePages.append(loader.load_and_split())


In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [36]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()
knowledgeStore = Chroma.from_documents(internalKbPages, embeddings) #, collection_name='internal-knowledge-base-1')

In [37]:
# result = knowledgeStore.similarity_search("When should we uninstall the app?", k=1)
#print(result)

# Vectorstore Knowledgebase Query Tool
The query hits the vector store

In [38]:
def internal_knowledgebase_search(query: str):
    result = knowledgeStore.similarity_search(query, k=1)        
    return  result

internal_knowledgebase_tool = Tool(
    name='Internal Knowledgebase tool', 
    func= internal_knowledgebase_search,    
    description="""
        Useful for querying about our internal services, teams and how to solve customer problems.
        The internal knowledge base has steps on how to solve customers problems, how our software systems work and which people and teams to reach out to. 
        Ask it for questions like 'What should I do?', 'What are the causes of this?', 'What do I do next?'
        """
)

# Setup agent

In [47]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent

tools = [incident_search_tool, internal_knowledgebase_tool]

#del memory 

# conversational agent memory
memory = ConversationBufferWindowMemory(memory_key='chat_history', k=3, return_messages=True)

# Set up the turbo LLM
turbo_llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

# create our agent
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [48]:
question = """
You are a support engineer who is helping troubleshoot software issues. 
You responses need to be specific and factful, they can only contain information that you get back from the tools and observations.
If you don't know the answer then say so and don't make up anything. 
If you have to reach out to a person or team then think about which is the best team for the problem and only return one team in your answer.

Question: My FX Prices are incorrect, who should I reach out to in order to fix them? 

"""
# Question: What did the user do to create the latest incident?
# Question: How do I troubleshoot a user who's application keeps crashing when they click detach?
# Question: My FX Prices are incorrect, who should I reach out to in order to fix them? 

answer = conversational_agent(question)
print(answer)
x = 1



> Entering new AgentExecutor chain...
{
    "action": "Internal Knowledgebase tool",
    "action_input": "Who should I reach out to for fixing incorrect FX prices?"
}
Observation: [Document(page_content='Teams and People  \nFX Infrastructure Team  \nHow to contact them? Send them an email to fxinfrateam@contoso.com In the email you must put the \nfolllowing information: How urgent this is, who to reach out to for more information  \n \nFX Data Team  \nHow to contact them? Cut a ServiceNow Incident against them with the following information: Service: \n"{name_of_service} {environment} Configuration Item: {name_of_service} {environment} Assignment \nGroup: FX Data Team  \n \nFX Dev Team  \nHow to contact them? Cut a ServiceNow Incident against them with the following informa tion: Service: \n"{name_of_service} {environment} Configuration Item: {name_of_service} {environment} Assignment \nGroup: FX Pricing Dev Team  \n \nCommunications Delivery Team  \nThis team manages relationships w